# Proyecto 1 - Deep Learning
## Estudiante: Allan Murillo González


## Instalación de dependencias

In [ ]:
!pip install numpy matplotlib torch torchvision

In [ ]:
import torch
import torchvision
#import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.nn.functional as F


## Parte 1: Preparación de Datos con Desbalanceo Inducido

In [ ]:
# Descargamos el conjunto de datos
dataset = CIFAR10(root='data/', download=True)
len(dataset)

In [ ]:
dataset_test = CIFAR10(root='data/', train=False)
len(dataset_test)

Abrir imagen:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
image, label = dataset[0]
plt.figure(figsize=(.90, .90))
plt.imshow(image)
print('Label:', label)

🐸

### 1. Cargar CIFAR-10 y dividir en entrenamiento (70%), validación (15%), prueba (15%).

In [ ]:
from torch.utils.data import random_split, Subset

# Divide in 70%,15% and 1%5

# train_ds, val_ds, test_ds = random_split(dataset, [0.7, 0.15,0.15])
# len(train_ds), len(val_ds), len(test_ds)

# Pero si tenemos test solo necesitamos divir uno en validación en 80% y 20
train_ds, val_ds = random_split(dataset, [0.80, 0.20])


test_ds = dataset_test

len(train_ds), len(val_ds),len(test_ds)


### 2. Desbalanceo Artificial: En el conjunto de entrenamiento, reducir las muestras de 3
Clases seleccionadas al 10% de su tamaño original (ej: aviones, barcos, ranas).

CIFAR-10 Clases:

| #                    | Clase |
|-----------------------------|--------------------|
| 0                    | Avión ✈️            |
| 1                    | Automóvil 🚗            |
| 2                    | Pájaro 🐦            |
| 3                    | Gato 🐱           |
| 4                    | Venado 🦌            |
| 5                    | Perro 🐶            |
| 6                    | Rana 🐸             |
| 7                    | Caballo 🐎             |
| 8                    | Barco ⛵            |
| 9                    | Camión 🚛            |







In [ ]:
# Clases a desbalancear: [0, 8, 6] (avión, barco, rana)
classes_to_reduce = [0, 8, 6]


In [ ]:
# Obtener las clases de cada muestra en train_ds
# train_labels = np.array([train_ds.dataset.targets[i] for i in train_ds.indices])
# print(train_labels)

from collections import Counter

# Crear lista con las clases
label_list = [labels for _, labels in train_ds]

# Contarlas
label_counts = Counter(label_list)

# Se orderna por ordern
sorted_counts = dict(sorted(label_counts.items()))

print(sorted_counts)


In [ ]:
# Obtener todos los índices y etiquetas de train_ds
train_indices = np.array(train_ds.indices)
train_labels = np.array([train_ds.dataset.targets[i] for i in train_indices])

# Contar muestras originales por clase
original_counts = Counter(train_labels)

# Seleccione los índices que desea conservar
new_indices = []
for cls in np.unique(train_labels):
    cls_indices = train_indices[train_labels == cls] # Obtener índices de esta clase

    if cls in classes_to_reduce:
        reduced_size = max(1, len(cls_indices) // 10)  # Mantener 10%
        cls_indices = np.random.choice(cls_indices, reduced_size, replace=False) # Seleccionar aleatoriamente

    new_indices.extend(cls_indices)

# Actualiza conjunto de datos reducido
train_ds_reduced = Subset(train_ds.dataset, new_indices)

# Imprimir nueva distribución de clases
new_labels = [train_ds.dataset.targets[i] for i in new_indices]
print(Counter(new_labels))

### 3. Aumento de Datos Básico: Solo para entrenamiento, aplicar normalización (utilice la media (0.5, 0.5, 0.5) y la std (0.5, 0.5, 0.5) ).

In [ ]:
import torchvision.transforms as transforms

# Definir una transformación para convertir imágenes PIL en tensores y normalizarlas
# Pytorch no sabe cómo trabajar con imágenes, debemos convertirlas a tensores.
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza con media y std
])


# Apply the transform to your datasets
train_ds_reduced.dataset.transform = transform
val_ds.dataset.transform = transform
#test_ds.dataset.transform = transform

### 4. Muestra un gráfico de barras con la distribución de clases antes/después del
desbalanceo.

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Get original labels from train_ds
original_counts = Counter([train_ds.dataset.targets[i] for i in train_ds.indices])  # Antes del desbalanceo

# Get new labels from the reduced train_ds (train_ds_reduced)
new_counts = Counter([train_ds_reduced.dataset.targets[i] for i in train_ds_reduced.indices])  # Después del desbalanceo

# Graficar
plt.figure(figsize=(12, 6))

# Subplot 1: Original distribution
plt.subplot(1, 2, 1)
plt.bar(original_counts.keys(), original_counts.values(), color='blue')
plt.title("Distribución Antes del Desbalanceo")
plt.xlabel("Clases")
plt.ylabel("Número de muestras")

# Subplot 2: Reduced distribution
plt.subplot(1, 2, 2)
plt.bar(new_counts.keys(), new_counts.values(), color='red')
plt.title("Distribución Después del Desbalanceo")
plt.xlabel("Clases")
plt.ylabel("Número de muestras")

# Mostrar gráfico
plt.tight_layout()
plt.show()


##Parte 2: Modelo Base Sobrealimentado (25%)

### 1. Arquitectura:
*  Red densamente conectada con mínimo 5 capas ocultas (ej: 3072 → 2048 →
1024 → 512 → 256 → 10).
*   Activaciones ReLU, dropout NO permitido en esta etapa.




Primero definamos nuestro dispositivo como el primer dispositivo cuda visible si tenemos CUDA disponible:

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

In [ ]:
from torch.utils.data import DataLoader
# Hiperparametros
input_size = 3072
num_classes = 10
epoch = 50
learning_rate = 0.01
batch_size = 128

train_loader = DataLoader(train_ds_reduced, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)
#test_loader = DataLoader(test_ds, batch_size)

In [ ]:
# Revisar si se puede hacer dentro de la clase de nn
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
def evaluate(model, loader, mode='train'):
    outputs = [model.validation_step(batch, mode=mode) for batch in loader]
    return model.validation_epoch_end(outputs, mode=mode)

In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(input_size, 2048)
        self.linear2 = nn.Linear(2048, 1024)
        self.linear3 = nn.Linear(1024, 512)
        self.linear4 = nn.Linear(512, 256)
        self.linear5 = nn.Linear(256, num_classes)

    def forward(self, xb):

        # Es el conjunto de datos CIFAR-10,
        # cada imagen con un tamano 32x32x3 (3072 pixels),
        # donde los 3 rpresentan los tres canales RGB.
        xb = xb.reshape(-1, input_size)
        # Ruido gaussiano
        #xb += torch.rand(1,784) * 0.8
        out = F.relu(self.linear1(xb))
        out = F.relu(self.linear2(out))
        out = F.relu(self.linear3(out))
        out = F.relu(self.linear4(out))
        out = self.linear5(out)
        return out

    def training_step(self, batch):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return loss

    # def train_validation_step(self, batch):
    #     images, labels = batch
    #     out = self(images)                    # Generar predictiones
    #     loss = F.cross_entropy(out, labels)   # Calcular loss
    #     acc = accuracy(out, labels)           # Calcular accuracy
    #     return {'train_loss': loss, 'train_acc': acc}

    def validation_step(self, batch, mode="train"):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {f'{mode}_loss': loss, f'{mode}_acc': acc}

    def validation_epoch_end(self, outputs, mode="train"):
        batch_losses = [x[f'{mode}_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Promedia los loss
        batch_accs = [x[f'{mode}_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Promedia accuracy
        return {f'{mode}_loss': epoch_loss.item(), f'{mode}_acc': epoch_acc.item()}


    def epoch_end(self, epoch, result, mode="train"):
        print(f"Epoch [{epoch}], {mode}_loss: {result[f'{mode}_loss']:.4f}, {mode}_acc: {result[f'{mode}_acc']:.4f}")

model = MnistModel()
model.to(device)

### 2. Entrenamiento:
* Optimizador: SGD con momentum (sin regularización L2).
* Función de pérdida: Cross-Entropy sin pesos de clases.
* Debe escoger las métricas para la evaluación.
* Entrenar por 100 épocas.

In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr, momentum=0.9)
    history_test = []
    history_train = []

    for epoch in range(epochs):

        # Entrenamiento
        for batch in train_loader:
             # Mover los datos al device
            images, labels = batch
            images = images.to(device)  # Mueve las imagenes
            labels = labels.to(device)  # Mueve las etiquetas

            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validación con muestras de trainig
        result_train = evaluate(model, train_loader)
        model.epoch_end(epoch, result_train)
        history_train.append(result_train)
        # Validación
        result = evaluate(model, val_loader, mode='val')
        model.epoch_end(epoch, result, mode='val')
        history_test.append(result)

    return history_train, history_test

In [ ]:
history_train, history_val = fit(epoch, learning_rate, model, train_loader, val_loader)

### 3. Análisis:
* Gráficas de pérdida y métricas (entrenamiento vs validación).
* Matriz de confusión para las 3 clases minoritarias.
* Explique: ¿Por qué el modelo probablemente tendrá bajo recall en las clases
minoritarias?

In [ ]:
def plot_accuracy(train_history, val_history):
    """
    Plots the accuracy of the training and validation datasets over epochs.

    Parameters:
    - train_history (list of dict): List containing training results for each epoch.
    - val_history (list of dict): List containing validation results for each epoch.
    """
    accuracies_train = [result['train_acc'] for result in train_history]
    accuracies_val = [result['val_acc'] for result in val_history]

    plt.plot(accuracies_train, '-x', label='Train Accuracy')
    plt.plot(accuracies_val, '-o', label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy (Train vs Validation) vs. Number of Epochs')
    plt.legend()
    plt.show()


In [ ]:
plot_accuracy(history_train,history_val)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Obtener predicciones reales y predichas del conjunto de validación
y_true = []
y_pred = []

model.eval()  # modo evaluación
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Filtramos solo las clases minoritarias [0, 8, 6]
minority_classes = [0, 8, 6]

mask = np.isin(y_true, minority_classes)
y_true_min = y_true[mask]
y_pred_min = y_pred[mask]

# Calcular matriz de confusión
cm = confusion_matrix(y_true_min, y_pred_min, labels=minority_classes)

# Mostrar matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f'{c}' for c in minority_classes])
disp.plot(cmap='Blues')
plt.title('Matriz de Confusión - Clases Minoritarias [0, 8, 6]')
plt.show()


**Explicación:** Como el dataset esta desvalanceado el modelo aprende muchísimo mejor a reconocer las clases mayoritarias (porque son las que más ve).

En cambio, las clases minoritarias (avión, barco y rana) aparecen tan pocas veces durante el entrenamiento, que el modelo no logra capturar bien sus patrones o características.

## Parte 3: Mitigación de Desbalanceo y Regularización (35%)

### 1. Pérdida Ponderada (10%)
  * Calcula pesos de clases inversamente proporcionales a su frecuencia.
  * Re-entrena el modelo base usando
    * CrossEntropyLoss(weight=class_weights).

In [ ]:
def calculate_class_weights(samples_per_class):
    total_samples = sum(samples_per_class)
    num_classes = len(samples_per_class)

    # Fórmula: total_samples / (num_classes * samples_per_class)
    class_weights = [total_samples / (num_classes * count) for count in samples_per_class]

    return torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
count_clasess = Counter(new_labels)
samples_per_class = list(count_clasess.values())
print(samples_per_class)

In [ ]:
class_weights = calculate_class_weights(samples_per_class)#.to(device)

In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(input_size, 2048)
        self.linear2 = nn.Linear(2048, 1024)
        self.linear3 = nn.Linear(1024, 512)
        self.linear4 = nn.Linear(512, 256)
        self.linear5 = nn.Linear(256, num_classes)

    def forward(self, xb):

        # Es el conjunto de datos CIFAR-10,
        # cada imagen con un tamano 32x32x3 (3072 pixels),
        # donde los 3 rpresentan los tres canales RGB.
        xb = xb.reshape(-1, input_size)
        # Ruido gaussiano
        #xb += torch.rand(1,784) * 0.8
        out = F.relu(self.linear1(xb))
        out = F.relu(self.linear2(out))
        out = F.relu(self.linear3(out))
        out = F.relu(self.linear4(out))
        out = self.linear5(out)
        return out

    def training_step(self, batch):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return loss

    # def train_validation_step(self, batch):
    #     images, labels = batch
    #     out = self(images)                    # Generar predictiones
    #     loss = F.cross_entropy(out, labels)   # Calcular loss
    #     acc = accuracy(out, labels)           # Calcular accuracy
    #     return {'train_loss': loss, 'train_acc': acc}

    def validation_step(self, batch, mode="train"):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {f'{mode}_loss': loss, f'{mode}_acc': acc}

    def validation_epoch_end(self, outputs, mode="train"):
        batch_losses = [x[f'{mode}_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Promedia los loss
        batch_accs = [x[f'{mode}_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Promedia accuracy
        return {f'{mode}_loss': epoch_loss.item(), f'{mode}_acc': epoch_acc.item()}


    def epoch_end(self, epoch, result, mode="train"):
        print(f"Epoch [{epoch}], {mode}_loss: {result[f'{mode}_loss']:.4f}, {mode}_acc: {result[f'{mode}_acc']:.4f}")

model = MnistModel()

In [ ]:
history_train, history_val = fit(epoch, learning_rate, model, train_loader, val_loader)

In [ ]:
plot_accuracy(history_train,history_val)

### 2. Técnicas Avanzadas (25%)
  * Dropout + BatchNorm: Añade capas de Dropout (p=0.5) y BatchNorm
después de cada capa oculta.
  * Aumento de Datos Agresivo: Rotaciones aleatorias (±30°), volteo horizontal,
ajustes de brillo/contraste.
  * Investigue sobre Early Stopping: Implemente una estrategia de early stopping
en el entrenamiento si la pérdida de validación no mejora en 10 épocas.

In [ ]:
class MnistModel(nn.Module):
    def __init__(self, dropout_prob=0.5):
      super().__init__()

      self.linear1 = nn.Linear(input_size, 2048)
      self.batchnorm1 = nn.BatchNorm1d(2048)  # BatchNorm after first layer
      self.dropout1 = nn.Dropout(dropout_prob)  # Dropout after first layer

      self.linear2 = nn.Linear(2048, 1024)
      self.batchnorm2 = nn.BatchNorm1d(1024)  # BatchNorm after second layer
      self.dropout2 = nn.Dropout(dropout_prob)  # Dropout after second layer

      self.linear3 = nn.Linear(1024, 512)
      self.batchnorm3 = nn.BatchNorm1d(512)  # BatchNorm after third layer
      self.dropout3 = nn.Dropout(dropout_prob)  # Dropout after third layer

      self.linear4 = nn.Linear(512, 256)
      self.batchnorm4 = nn.BatchNorm1d(256)  # BatchNorm after fourth layer
      self.dropout4 = nn.Dropout(dropout_prob)  # Dropout after fourth layer

      self.linear5 = nn.Linear(256, num_classes)

    def forward(self, xb):
      # Es el conjunto de datos CIFAR-10,
      # cada imagen con un tamano 32x32x3 (3072 pixels),
      # donde los 3 rpresentan los tres canales RGB.
      xb = xb.reshape(-1, input_size)

      out = F.relu(self.linear1(xb))
      out = self.batchnorm1(out)  # Apply BatchNorm
      out = self.dropout1(out)  # Apply Dropout

      out = F.relu(self.linear2(out))
      out = self.batchnorm2(out)  # Apply BatchNorm
      out = self.dropout2(out)  # Apply Dropout

      out = F.relu(self.linear3(out))
      out = self.batchnorm3(out)  # Apply BatchNorm
      out = self.dropout3(out)  # Apply Dropout

      out = F.relu(self.linear4(out))
      out = self.batchnorm4(out)  # Apply BatchNorm
      out = self.dropout4(out)  # Apply Dropout

      out = self.linear5(out)
      return out

    def training_step(self, batch):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return loss

    # def train_validation_step(self, batch):
    #     images, labels = batch
    #     out = self(images)                    # Generar predictiones
    #     loss = F.cross_entropy(out, labels)   # Calcular loss
    #     acc = accuracy(out, labels)           # Calcular accuracy
    #     return {'train_loss': loss, 'train_acc': acc}

    def validation_step(self, batch, mode="train"):
        images, labels = batch
        images = images.to(next(self.parameters()).device)
        labels = labels.to(next(self.parameters()).device)

        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {f'{mode}_loss': loss, f'{mode}_acc': acc}

    def validation_epoch_end(self, outputs, mode="train"):
        batch_losses = [x[f'{mode}_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Promedia los loss
        batch_accs = [x[f'{mode}_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Promedia accuracy
        return {f'{mode}_loss': epoch_loss.item(), f'{mode}_acc': epoch_acc.item()}


    def epoch_end(self, epoch, result, mode="train"):
        print(f"Epoch [{epoch}], {mode}_loss: {result[f'{mode}_loss']:.4f}, {mode}_acc: {result[f'{mode}_acc']:.4f}")

model = MnistModel()

In [ ]:
from torchvision import transforms

# Transformaciones para entrenamiento
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),                 # Volteo Horizontal
    transforms.RandomRotation(30),                     # Rotaciones entre -30° y 30°
    transforms.ColorJitter(                            # Cambios de brillo, contraste, saturación
        brightness=0.2, contrast=0.2, saturation=0.2
    ),
    transforms.ToTensor(),                             # Lo pasa a tensor
    transforms.Normalize((0.5, 0.5, 0.5),              # Normalización
                         (0.5, 0.5, 0.5))
])

train_ds_reduced.dataset.transform = transform
val_ds.dataset.transform = transform


In [ ]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True):
        self.patience = patience        # Cuántas épocas esperar
        self.best_loss = float('inf')   # La mejor pérdida encontrada
        self.counter = 0                # Cuántas épocas sin mejorar
        self.verbose = verbose          # Mostrar mensajes o no

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss   # Nueva mejor pérdida
            self.counter = 0            # Reiniciar contador
        else:
            self.counter += 1           # No mejoró → contar
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} / {self.patience}")

        return self.counter >= self.patience


In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr, momentum=0.9)
    history_test = []
    history_train = []

    early_stopping = EarlyStopping(patience=10)

    for epoch in range(epochs):
        # Entrenamiento
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Evaluar entrenamiento y validación
        result_train = evaluate(model, train_loader)
        result_val = evaluate(model, val_loader, mode='val')

        model.epoch_end(epoch, result_train)
        model.epoch_end(epoch, result_val, mode='val')

        history_train.append(result_train)
        history_test.append(result_val)

        # Checar EarlyStopping
        if early_stopping(result_val['val_loss']):
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

    return history_train, history_test


In [ ]:
# Lo estaba corriendo nuevamente y no me dio tiempo :p
history_train, history_val = fit(epoch, learning_rate, model, train_loader, val_loader)

### 3. Análisis:
  * Compara las curvas de entrenamiento vs las del modelo base.
  * ¿Cómo afectó BatchNorm a la velocidad de convergencia? (Justifique con
métricas).
  * Para una clase minoritaria, calcule la mejora en F1-score tras aplicar los
pesos.

In [ ]:
plot_accuracy(history_train,history_val)

## Parte 4: Evaluación Final y Métricas (20%)

### 1. Métricas en Test:
  * Reporte precisión, recall y F1-score por clase.


In [ ]:
from sklearn.metrics import classification_report
import torch


In [ ]:
def get_all_preds(model, loader):
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, preds = torch.max(outputs, dim=1)  # Clase con mayor probabilidad
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


In [ ]:
labels, preds = get_all_preds(model, train_loader)

print(classification_report(labels, preds, target_names=dataset.classes))


  * Analice los resultados obtenidos.


A partir de las métricas de precisión (precision), exhaustividad (recall) y F1-score obtenidas para cada clase, se observa lo siguiente:

Las clases más balanceadas o mayoritarias (como automobile, horse, ship) tienden a tener mejores resultados, con F1-scores entre 0.53 y 0.60.

Las clases minoritarias (como airplane, ship y frog, que fueron desbalanceadas a propósito) presentan resultados considerablemente inferiores en comparación con las clases mayoritarias.

Además, algunas clases como bird y cat, a pesar de no ser minoritarias, presentan métricas particularmente bajas, probablemente por su alta similitud visual con otras clases.

  * Seleccione la clase con peor performance y proponga una estrategia
  específica para mejorarla.


Analizando el F1-score por clase, la peor performance se observa en la clase bird, con un F1-score de 0.24 (el más bajo de todas las clases).

* Clase	F1-score
* bird	0.24
Esto se debe a:

Precision baja (0.39): el modelo predice bird muchas veces cuando no corresponde.

Recall muy bajo (0.17): de todas las imágenes reales de bird, solo logra identificar correctamente un 17%.

Como el mayor problema es el recall bajo (es decir, el modelo no está reconociendo bien las imágenes de bird), se puede proponer:

Estrategia: Aumento de Datos Específico (Data Augmentation)
Consiste en generar más ejemplos sintéticos de bird mediante transformaciones sobre las imágenes existentes:

* Rotaciones

* Zoom in/out

* Cambios de brillo/contraste

* Volteo horizontal/vertical

* Recortes aleatorios

* Distorsiones pequeñas

Links:
* https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/cifar10_tutorial.ipynb

* https://christianbernecker.medium.com/how-to-create-a-confusion-matrix-in-pytorch-38d06a7f04b7
